# Train训练

In [ ]:
#!cd yolov5

In [ ]:
#!cd yolov5
#!python train.py --img 640 --batch-size -1 --epochs 500 --data ../garbage.yaml --weights yolov5s.pt

In [2]:
# 将results.csv变成图片的格式
%cd yolov5
from utils.plots import plot_results
plot_results('runs/train/exp/results.csv')

/home1/wckdata/garbage_detect/yolov5


<img src="/home1/wckdata/garbage_detect/yolov5/runs/train/exp/results.png" width="50%">

# 验证val中的图片

In [ ]:
#!cd yolov5
!python detect.py --weights runs/train/exp/weights/best.pt --img 640 --conf 0.25 --source ../datasets/DLLG_YOLO/val/images/

In [ ]:
import os
import glob
from IPython.display import display, Image

directory_path="/home1/wckdata/garbage_detect/yolov5/runs/detect/exp/"
file_count=len([name for name in os.listdir(directory_path)])
print(f'文件夹 "{directory_path}" 中的文件数量是: {file_count}')

#display inference on some test images
images = glob.glob('/home1/wckdata/garbage_detect/yolov5/runs/detect/exp/*.jpg')

for imageName in images[:3]: #assuming JPG
    display(Image(filename=imageName, width=400))

# 视频抽帧

In [ ]:
# 导入所有必要的库
import cv2
import os
# 从指定的路径读取视频
cam = cv2.VideoCapture("./test_video/785124.mp4")

if not os.path.exists("./test_video/img"):
     os.mkdir("./test_video/img")

  
# frame
currentframe = 0
  
while 1:
    ret, frame = cam.read()
    if ret:
        name = "./test_video/img/" + str(currentframe)+".jpg"
        print(f"create {name}")
        cv2.imwrite(name, frame)
        currentframe += 100
    else:
        break
  
# 一旦完成释放所有的空间和窗口
cam.release()
cv2.destroyAllWindows()

# model处理一张图片

In [1]:
import argparse
import csv
import os
import platform
import sys
import shutil
from pathlib import Path

import pyrealsense2 as rs
import numpy as np
import cv2
import random
import torch
import torch.backends.cudnn as cudnn # cuda模块
import time

ROOT = os.path.join("/home1/wckdata/garbage_detect/", "yolov5")
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
print(str(ROOT))
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

from yolov5.utils.torch_utils import select_device
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import (
    check_img_size, non_max_suppression, apply_classifier, scale_boxes,
    xyxy2xywh, strip_optimizer, set_logging, Profile, LOGGER
    )
from yolov5.utils.augmentations import letterbox
from ultralytics.utils.plotting import Annotator, colors, save_one_box
from yolov5.models.experimental import attempt_load

/home1/wckdata/garbage_detect/yolov5


In [2]:
device = select_device('0')
model = attempt_load('./best_train/20240709best.pt', device=device)
imgsz = check_img_size(640, s=model.stride.max()) # 验证图片大小
model.half()
cudnn.benchmark = True

YOLOv5 🚀 2024-7-4 Python-3.8.19 torch-1.11.0+cu113 CUDA:0 (NVIDIA A40-12Q, 12099MiB)

Fusing layers... 
Model summary: 157 layers, 7018216 parameters, 0 gradients, 15.8 GFLOPs


In [3]:
read_image = cv2.imread("/home1/wckdata/garbage_detect/test_video/img/17500.jpg") #(h,w,bgr)
color_image = np.asanyarray(read_image)
# print(color_image.shape) #(480, 640, 3)
imgs = [None]
imgs[0] = color_image
im0s = imgs.copy()
img = [letterbox(x, new_shape=640)[0] for x in im0s]  # img: 进行resize + pad之后的图片 list格式
img = np.stack(img, 0)  #沿着0dim进行堆叠 此时shape已经是[batch_size, h, w, channel]
# print(img.shape) # (1, 480, 640, 3)
img = img[:, :, :, ::-1].transpose(0, 3, 1, 2)  # BGR to RGB, to 3x416x416, uint8 to float32
# print(img.shape) # (1, 3, 480, 640)
img = np.ascontiguousarray(img, dtype=np.float16 if True else np.float32)
                # ascontiguousarray函数将一个内存不连续存储的数组转换为内存连续存储的数组，使得运行速度更快。
img /= 255.0  # 0 - 255 to 0.0 - 1.0
# print(img.shape) # (1, 3, 480, 640)
img = torch.from_numpy(img).to(device) #将numpy转为pytorch的tensor,并转移到运算设备上计算

In [4]:
pred = model(img)[0]
print(pred)

tensor([[[8.57031e+00, 6.50781e+00, 1.50156e+01,  ..., 3.35999e-02, 4.18945e-01, 2.04346e-01],
         [1.49688e+01, 8.68750e+00, 2.86719e+01,  ..., 2.45209e-02, 2.97363e-01, 3.01758e-01],
         [1.80156e+01, 7.93750e+00, 3.38750e+01,  ..., 3.70483e-02, 2.18018e-01, 3.78906e-01],
         ...,
         [5.70000e+02, 4.55000e+02, 2.09250e+02,  ..., 7.60742e-01, 1.92261e-01, 4.30298e-02],
         [5.91000e+02, 4.53250e+02, 1.65250e+02,  ..., 6.31836e-01, 2.65381e-01, 7.34253e-02],
         [6.15000e+02, 4.54000e+02, 1.93750e+02,  ..., 4.43359e-01, 3.60596e-01, 1.75903e-01]]], device='cuda:0', dtype=torch.float16)


In [5]:
pred = non_max_suppression(pred, 0.25, 0.45)
print(pred)

[tensor([[3.46750e+02, 2.11750e+02, 4.23750e+02, 2.74750e+02, 4.29443e-01, 1.00000e+00],
        [2.66500e+02, 1.92750e+02, 3.55500e+02, 2.59500e+02, 2.98340e-01, 0.00000e+00]], device='cuda:0')]


In [6]:
def plot_one_box(x, img, color=None, label=None, line_thickness=None):
    tl = line_thickness or round(0.002 * (img.shape[0] + img.shape[1]) / 2) + 1  # 字体大小
    color = color or [random.randint(0, 255) for _ in range(3)]	#随机颜色
    c1, c2 = (int(x[0]), int(x[1])), (int(x[2]), int(x[3]))
    cv2.rectangle(img, c1, c2, color, thickness=tl, lineType=cv2.LINE_AA)   #通过确定对角线位置来画矩形框
    if label:
        tf = max(tl - 1, 1)  # font thickness字体粗细
        t_size = cv2.getTextSize(label, 0, fontScale=tl / 3, thickness=tf)[0]
        c2 = c1[0] + t_size[0], c1[1] - t_size[1] - 3
        cv2.rectangle(img, c1, c2, color, -1, cv2.LINE_AA)  # filled
        cv2.putText(img, label, (c1[0], c1[1] - 2), 0, tl / 3, [225, 255, 255], thickness=tf, lineType=cv2.LINE_AA)
                    #(图片img,“文本内容”,(左下角坐标),字体,字体大小,(颜色)，线条粗细，线条类型)

In [7]:
names = model.names
colors = [[random.randint(0, 255) for _ in range(3)] for _ in range(len(names))]
label_img = ""
for i, det in enumerate(pred):
    im0 = im0s[i].copy()
    if len(det):
        det[:, :4] = scale_boxes(img.shape[2:], det[:, :4], im0.shape).round()
        for c in det[:, 5].unique():
            n = (det[:, 5] == c).sum()  # detections per class
            s = f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string
        for *xyxy, conf, cls in reversed(det):
            c = int(cls)  # integer class
            label = f"{names[c]}"
            confidence = float(conf)
            confidence_str = f"{confidence:.2f}"
            label_img = f"{label}:{confidence_str}"
            plot_one_box(xyxy, im0, label=label_img, color=colors[int(cls)], line_thickness=3)


In [9]:
#cv2.imwrite("/home1/wckdata/garbage_detect/test_video/img/result.jpg", im0)